In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score, confusion_matrix
from sklearn.manifold import TSNE
from scipy.optimize import linear_sum_assignment



In [ ]:

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)

all_labels = np.array(train_data.targets)
all_images = train_data.data.unsqueeze(1).float() / 255.0






In [ ]:

class CNNEncoder(nn.Module):
    def __init__(self, embedding_dim=64):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7, 128),
            nn.ReLU(),
            nn.Linear(128, embedding_dim)
        )

    def forward(self, x):
        x = self.encoder(x)
        return self.fc(x)



In [5]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
encoder = CNNEncoder(embedding_dim=64).to(device)
clf_head = nn.Linear(64, 10).to(device)

optimizer = optim.Adam(list(encoder.parameters()) + list(clf_head.parameters()), lr=1e-3)
criterion = nn.CrossEntropyLoss()

for epoch in range(15):
    total_loss = 0
    encoder.train()
    clf_head.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        feats = encoder(images)
        logits = clf_head(feats)
        loss = criterion(logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Classification Loss: {total_loss/len(train_loader):.4f}")





Epoch 1, Classification Loss: 0.5004
Epoch 2, Classification Loss: 0.3077
Epoch 3, Classification Loss: 0.2617
Epoch 4, Classification Loss: 0.2311
Epoch 5, Classification Loss: 0.2037
Epoch 6, Classification Loss: 0.1813
Epoch 7, Classification Loss: 0.1642
Epoch 8, Classification Loss: 0.1438
Epoch 9, Classification Loss: 0.1298
Epoch 10, Classification Loss: 0.1130
Epoch 11, Classification Loss: 0.0960
Epoch 12, Classification Loss: 0.0824
Epoch 13, Classification Loss: 0.0731
Epoch 14, Classification Loss: 0.0612
Epoch 15, Classification Loss: 0.0548


In [ ]:

encoder.eval()
with torch.no_grad():
    feats = encoder(all_images.to(device)).cpu().numpy()
labels = all_labels


In [ ]:

class DEC(nn.Module):
    def __init__(self, embedding_dim=64, n_clusters=10):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.n_clusters = n_clusters
        self.cluster_centers = nn.Parameter(torch.randn(n_clusters, embedding_dim))
        torch.nn.init.xavier_uniform_(self.cluster_centers.data)

    def forward(self, z):
        q = 1.0 / (1.0 + torch.sum((z.unsqueeze(1) - self.cluster_centers)**2, dim=2))
        q = q ** ((1 + 1.0) / 2.0)
        q = (q.t() / torch.sum(q, dim=1)).t()
        return z, q



In [ ]:

features_tensor = torch.tensor(feats, dtype=torch.float32).to(device)
dec_model = DEC(embedding_dim=64, n_clusters=10).to(device)

kmeans = KMeans(n_clusters=10, n_init=20)
preds_init = kmeans.fit_predict(feats)
dec_model.cluster_centers.data.copy_(
    torch.tensor(kmeans.cluster_centers_, dtype=torch.float32).to(device)
)


In [ ]:

def target_distribution(q):
    weight = q ** 2 / torch.sum(q, dim=0)
    return (weight.t() / torch.sum(weight, dim=1)).t()

optimizer = optim.Adam(dec_model.parameters(), lr=1e-3)
criterion = nn.KLDivLoss(reduction='batchmean')

for epoch in range(30):
    dec_model.train()
    _, q = dec_model(features_tensor)
    p = target_distribution(q).detach()
    loss = criterion((q + 1e-10).log(), p)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, KL Loss: {loss.item():.6f}")




In [ ]:

dec_model.eval()
with torch.no_grad():
    z_final, q_final = dec_model(features_tensor)
    pred_labels = q_final.cpu().numpy().argmax(axis=1)

conf_matrix = confusion_matrix(labels, pred_labels)
row_ind, col_ind = linear_sum_assignment(-conf_matrix)
mapped_preds = np.zeros_like(pred_labels)
for i in range(len(row_ind)):
    mapped_preds[pred_labels == col_ind[i]] = row_ind[i]

accuracy = np.mean(mapped_preds == labels)
print(f"Clustering Accuracy: {accuracy * 100:.2f}%")
print("Silhouette:", silhouette_score(z_final.cpu(), pred_labels))
print("CH Index:", calinski_harabasz_score(z_final.cpu(), pred_labels))
print("DB Index:", davies_bouldin_score(z_final.cpu(), pred_labels))




In [ ]:

sample_indices = np.random.choice(len(z_final), size=3000, replace=False)
z_sample = z_final[sample_indices].cpu().numpy()
label_sample = mapped_preds[sample_indices]

z_tsne = TSNE(n_components=2, n_iter=500, random_state=42).fit_transform(z_sample)
plt.figure(figsize=(8, 6))
plt.scatter(z_tsne[:, 0], z_tsne[:, 1], c=label_sample, cmap='tab10', s=10)
plt.title("t-SNE Clustering Visualization")
plt.colorbar()
plt.show()

